# BMDS for ToxRefDB - Part 1 - Preparing data from 
The objectives of this document is to outline the amount of data that may be available for an automated BMD workflow, and to further identify the potential caveats in an automated BMD analysis of these data.

## Filtering ToxRefDB to obtain the studies available for consideration  
* Database version: ToxRefDB_2_0 (dev_toxrefdb_2_0 on mysql-res1.epa.gov)
* Quality filters using columns in the database:
    + data_usability = acceptable
    + Number of Dose = control + >2 treatment dose
* Filter Flags 
    + Missing Values
        + One or more sample size in a dataset
        + One or more effect value in a dataset
        + One or more dose value in a dataset
        + One or more dose level in a dataset
    + Duplicated Data
        + Dose value is the same for more than one dose level in a dataset
    + Zero(s)  
        + sample size is zero 
* Possible issues with the BMD models:
    + Some have small sample size per dose group (e.g., dog studies have <10 animals per dose group) 
    + Not all effect value or effect value units can be used 
    + Some of the effect values refer to the same biological parameter (i.e. body weight measured as absolute unit or as a percentage) 
    + Effect values may not not have sample size for one or more dose level


In [1]:
library(RMySQL)
library(data.table)
library(magrittr)

Loading required package: DBI


In [4]:
setwd("--")

In [5]:
con <- dbConnect(drv = RMySQL::MySQL(), group = "toxrefdb_2_0")

toxRef <- dbGetQuery(con, "SELECT 
          study.study_id,
          study.chemical_id,
          study.dsstox_gsid,
          study.study_type,
          study.study_year,
          study.study_source,
          study.species,
          study.strain_group,
          study.admin_route,
          study.admin_method,
          study.substance_purity,
          endpoint.endpoint_category,
          endpoint.endpoint_type,
          endpoint.endpoint_target,
          endpoint.endpoint_id,
          tg_effect.life_stage,
          effect.effect_id,
          effect.effect_desc,
          tg_effect.effect_id,
          tg_effect.tg_effect_id,
          tg.sex,
          tg.generation,
          dose.dose_level,
          dtg.dose_adjusted,
          dtg.dose_adjusted_unit,
          dtg_effect.sample_size,
	      dtg_effect.effect_val,
	      dtg_effect.effect_val_unit,
          dtg_effect.effect_var,
          dtg_effect.effect_var_type,  
          dtg_effect.treatment_related,
          dtg_effect.critical_effect
      FROM 
          ((((((((study INNER JOIN dose ON dose.study_id=study.study_id)
              INNER JOIN tg ON tg.study_id=study.study_id)
                  INNER JOIN dtg ON tg.tg_id=dtg.tg_id AND dose.dose_id=dtg.dose_id)
                      INNER JOIN tg_effect ON tg.tg_id=tg_effect.tg_id)
                          INNER JOIN dtg_effect ON tg_effect.tg_effect_id=dtg_effect.tg_effect_id AND dtg.dtg_id=dtg_effect.dtg_id)
                              INNER JOIN effect ON effect.effect_id=tg_effect.effect_id)
                                  INNER JOIN endpoint ON endpoint.endpoint_id=effect.endpoint_id)
                                    INNER JOIN obs ON obs.study_id=study.study_id AND obs.endpoint_id=endpoint.endpoint_id)
      WHERE study.data_usability='acceptable' ") %>% 
  data.table() 

toxRef[ , dose_no := max(dose_level), by = study_id]


# Clean data for BMDS

*Which problems can we adress?*

  * New colums are created to show what data are adjusted and the new values associatd with it
    * effect_val_adj
    * effect_val_unit_adj
    * effect_var_adj
    * effect_var_type_adj

What kind of effect units can we use?

There are about 1,598 unique units used in the database. 

__Count data to be used for a Dichotomous Model__

* Find and convert all variant of a unit
    *"incidence",
"incindence",
"Incidence",
"inicidence",
"total incidence",
"fetal incidence",
"litter incidence",
"percent incidence",
'incidnece',
"incidences",
"indidence",
"incidental",
"incidnet",
"incident",
"incidene",
"incidents",
"fetal incicdence",
"incidence", 
" incidence",
"incidnce",
"incidence/animals alive",
"incidence of early resorptions",
"incincidence",
"incidnence" 

* Converting Percent incidence to incident

$$ \frac{n*pinc}{100} $$

n = sample size  
pinc = percent incidence

__Converting variance estimation for continuous data__

* Converting standard error to standard deviation

$$ sd=se*\sqrt{n} $$
sd = standard deviation  
se = standard error  
n = sample size  

* Converting 95% confidence interval to standard error

$$ se=\frac{Y_u-Y_l}{2*t_{df=n-1}} $$
se = standard error  
$Y_u$ = upper bound of the mean  
$Y_l$ = lower bound of the mean  
t = student t-distribution  
df = degrees of freedom   


In [6]:

trCorrection <- copy(toxRef) %>% .[dose_no > 2] # at least 3 treatment related doses + control
trCorrection[ , sample_size := as.numeric(sample_size)]

##Different variations of incidence in ToxRefDB

inc <- c( "incidence",
          "#  of dams with effected pups",
          "# dead pups",
          "# litters",
          "# of animals affected",
          "# of dam with effected pups",
          "# of dams with affected pups",
          "# of Dams with effected pups",
          "# of dams with effected pups",
          "# of females with resorptions",
          "# premature deaths",
          "#of dams with affected pups",
          "fetal incicdence",
          "fetal incidence",
          "incidence",
          "Incidence",
          " incidence", 
          "incidences",
          "incidene",
          "incident",
          "incidental",
          "incidents",
          "incidnce",
          "incidnece",
          "incidnence",
          "incidnet",
          "incincidence",
          "incindence",
          "indidence",
          "inicidence",
          "litter incidence",
          "number of fetuses affected",
          "total # anomalies",
          "total # corpora lutea",
          "total # malformations",
          "total incidence" )

inc_per <- c("% fetal incidence",
             "% incidence",
             "% litter incidence",
             "percent incidence" )

##Data 
trCorrection[effect_val_unit %in% inc_per , effect_val_adj := (sample_size * effect_val)/100]
trCorrection[effect_val_unit %in% inc_per , effect_val_unit_adj := "incidence"]
trCorrection[effect_val_unit %in% inc, effect_val_unit_adj := "incidence"]
trCorrection[effect_val_unit == "incidence", effect_val_adj := effect_val]
trCorrection[is.na(effect_val_adj), effect_val_adj := effect_val]

## Correction of Continuous Data
var_sd <- c("SD", "sd", "std dev", "Sd", "standard deviation")
var_se <- c("SE", "std error", "SE " )
var_95 <- c( "95% confidence limit" , "95% +/- confidence limit")

trCorrection[effect_var_type %in% var_sd, effect_var_type_adj := "sd"]
trCorrection[effect_var_type %in% var_sd, effect_var_adj := effect_var]
trCorrection[effect_var_type %in% var_se, effect_var_type_adj := "sd"]
trCorrection[effect_var_type %in% var_se, effect_var_adj := round(effect_var*sqrt(sample_size),2)]
trCorrection[effect_var_type %in% var_95, effect_var_temp := (effect_var*2)/(2*qt(0.95, df=sample_size-1))]
trCorrection[effect_var_type %in% var_95, effect_var_adj := round(effect_var_temp*sqrt(sample_size),2)]
trCorrection[effect_var_type %in% var_95, effect_var_type_adj := "sd"]

trCorrection[ , id := paste(study_id, endpoint_id, tg_effect_id, sex, sep = "_")]


Warning message in eval(jsub, SDenv, parent.frame()):
"NAs introduced by coercion"

In [7]:
length(unique(trCorrection$id))

[1] 92645

## Dichotomus Data  

We can understand how BMD dichotomous data with interpretable incidence units. It is likely that not all of the studies in this table will be BMD-amenable after considering the possible issues; this is the maximum set.

* Possible issues in the dataset:
    + If controls are not available, should we use zero incidence? This may be misleading, particularly with the use of historical control data.
    + Effect_val : some have fractional data, even though incidence data should be whole numbers. These observations may need to be dropped. 
      + example: lymph node granuloma has a 2.2 effect unit
      
There are two types of Dicotomous Endpoints (1) Cancer and (2) Non-Cancer   
      

In [8]:
# Below are the endpoints that we define and separate as cancer enpoints 
cancer <- c("adenocarcinoma", 
            "hemangioma",             
            "hemangiosarcoma", 
            "adenoma",  
            "adenoma/carcinoma combined",                  
            "neoplasm nos",                                  
            "carcinoma",                                     
            "papilloma",                           
            "mass",                                         
            "nodule(s)",                                    
            "interstitial cell tumor benign",              
            "hematopoietic cell proliferation erythrocytic", 
            "eosinophilic focus",     
            "foci",          
            "luteoma",
            "transitional epithelial carcinoma", 
            "fibrosarcoma",                                     
            "pheochromocytoma benign",                      
            "pheochromocytoma nos",                              
            "leukemia",                                                                       
            "fibroadenoma",          
            "lymphoma malignant",                                
            "lymphoma nos",                                     
            "carcinoma nos",                                
            "mesothelioma benign",                               
            "basophilic focus",                                  
            "mixed tumor nos",                                   
            "granuloma",
            "neoplastic nodule",                                
            "polyp stromal",     
            "hepatocellular carcinoma",                          
            "keratoacanthoma",                               
            "fibroma",
            "mixed tumor malignant",                             
            "interstitial cell tumor nos",                       
            "palpable mass",                                     
            "cystadenoma",                                       
            "adenomatosis",                                 
            "hepatocellular adenoma",                            
            "dysplasia" )

In [9]:
dataCancer <- trCorrection[effect_val_unit_adj == "incidence" ] %>% 
  .[effect_desc %in% cancer] %>%
  .[ , id_count := .N, by = id] %>%
  .[ id_count >= 4] %>%
  .[ , na.ns:=max(is.na(sample_size)), by=id] %>% # identify dataset where sample size is missing for some dose  
  .[ , na.inc:=max(is.na(effect_val_adj)), by=id] %>% # identify dataset where incidence data is missing for some dose
  .[ , dose.unq:=max(is.na(dose_adjusted)), by=id] %>% # identify dataset with missing dose value
  .[ , dose_dup:=max(duplicated(dose_adjusted)), by=id] %>% # identify dataset with duplicated dose
  .[ , all_dose_in_study:=(dose_no==(id_count-1))] %>% # identify dataset where all dose level are included
  .[ , eff_val_na:=max(is.na(effect_val)), by=id] %>% # identify datasets with effect value missing
  .[ , sample_size_all := min(sample_size), by=id] %>% # identify smallest sample size
  .[ eff_val_na==0] %>%  
  .[ na.ns==0] %>% 
  .[ na.inc==0] %>%
  .[ dose.unq==0] %>%
  .[ all_dose_in_study==TRUE] %>%
  .[ dose_dup==0] %>% 
  .[ sample_size_all > 0]

setnames(dataCancer, 
         c("dose_adjusted", "sample_size","effect_val_adj"), 
         c("doses", "ns", "incidences"))

In [10]:
cancer_tr <- dataCancer[ , list(id, doses, ns, incidences) ]%>%
  aggregate( . ~id, data = . , paste, collapse = ",") %>% 
  data.table() %>%
  .[ , c("study_id", "endpoint_id", "tg_effect_id", "sex"):=tstrsplit(id, "_", fixed=TRUE)]

length(unique(cancer_tr$id))

## Write data to be sent to Notebook for BMDS processing 
write.csv(cancer_tr,"cancer_tr.csv")


[1] 1180

### Non-Cancer Dicotomous Endpoints

In [11]:
dataNoCancer <- trCorrection[effect_val_unit_adj == "incidence"] %>% 
  .[!(effect_desc %in% cancer)] %>%
  .[ , id_count := .N, by = id] %>%
  .[ id_count >= 4] %>%
  .[ , na.ns:=max(is.na(sample_size)), by=id] %>% # identify dataset where sample size is missing for some dose  
  .[ , na.inc:=max(is.na(effect_val_adj)), by=id] %>% # identify dataset where incidence data is missing for some dose
  .[ , dose.unq:=max(is.na(dose_adjusted)), by=id] %>% # identify dataset with missing dose value
  .[ , dose_dup:=max(duplicated(dose_adjusted)), by=id] %>% # identify dataset with duplicated dose
  .[ , all_dose_in_study:=(dose_no==(id_count-1))] %>% # identify dataset where all data dose level are included
  .[ , eff_val_na:=max(is.na(effect_val)), by=id] %>% # identify datasets with effect value missing
  .[ , sample_size_all := min(sample_size), by=id] %>% # identify smallest sample size
  .[ eff_val_na==0] %>%  
  .[ na.ns==0] %>% 
  .[ na.inc==0] %>%
  .[ dose.unq==0] %>%
  .[ all_dose_in_study==TRUE] %>%
  .[ dose_dup==0] %>% 
  .[ sample_size_all > 0]

In [12]:
setnames(dataNoCancer, 
         c("dose_adjusted", "sample_size","effect_val"), 
         c("doses", "ns", "incidences"))

dataNoCancer[ , id_count := .N, by = id]

dichotomous_tr <- dataNoCancer[ , list(id, doses, ns, incidences) ]%>%
  aggregate( . ~ id, data = . , paste, collapse=",") %>% 
  data.table() %>%
  .[ , c("study_id", "endpoint_id", "tg_effect_id", "sex"):=tstrsplit(id, "_", fixed=TRUE)]

## Write data to be sent to Notebook for BMDS processing 
write.csv(dichotomous_tr,"dichotomous_tr.csv")

In [13]:
length(unique(dichotomous_tr$id))

[1] 17360

Dichotomous data file is really large.  If computer resource or time is a problem, break the data into 4 section to run.

In [14]:
dichotomous1 <- data.table(dichotomous_tr) %>%
 .[0:5000]

write.csv(dichotomous1,"dichotomous_tr1.csv")

dichotomous2 <- data.table(dichotomous_tr) %>%
 .[5001:10000]

write.csv(dichotomous2,"dichotomous_tr2.csv")

dichotomous3 <- data.table(dichotomous_tr) %>%
 .[10001:15000]

write.csv(dichotomous3,"dichotomous_tr3.csv")

dichotomous4 <- data.table(dichotomous_tr) %>%
 .[15001:17190]

write.csv(dichotomous4,"dichotomous_tr4.csv")

## Continuous Data  

To perform BMDs for continuous data, variance estimation per dose group is needed. The availability of this information will be a large limiter on the ability to run an automated BMDS workflow. It is likely that not all of the studies in this table will be BMD-amenable after considering the possible issues; this is the maximum set. 

* Possible Issues:
  + BMD not optimized to model effects with neg dose-response (i.e. body weigh going down)
  + Many different effect_desc and effect_val_unit for one endpoint_target
  + Example: Body weight
  + Effect_desc
    * "body weight"
    * "body weight gain"
  + Effect_val_unit
    * "g"
    * "kg"
    * "g/animal"
    * "kg/dog/week"
    * "%"
    * "None"
    * "kg/day"
    * "g "
    * "g/animal/day"
    * "% body weight gain"
    * "% increase"
    * "g/mouse"
    * "g/rat"
    * "grams"
    * "gram"

In [14]:
dataContinuous <- trCorrection[!is.na(effect_var_adj)] %>%
  .[!is.na(effect_var_type_adj)] %>%
  .[ , id_count := .N, by = id] %>%
  .[ id_count >= 4] %>%
  .[ , na.ns:=max(is.na(sample_size)), by=id] %>% # identify dataset where sample size is missing for some dose  
  .[ , na.inc:=max(is.na(effect_val_adj)), by=id] %>% # identify dataset where incidence data is missing for some dose
  .[ , dose.unq:=max(is.na(dose_adjusted)), by=id] %>% # identify dataset with missing dose value
  .[ , dose_dup:=max(duplicated(dose_adjusted)), by=id] %>% # identify dataset with duplicated dose
  .[ , all_dose_in_study:=(dose_no==(id_count-1))] %>% # identify dataset where all data dose level are included
  .[ , eff_val_na:=max(is.na(effect_val)), by=id] %>% # identify datasets with effect value missing
  .[ , sample_size_all := min(sample_size), by=id] %>% # identify smallest sample size
  .[ eff_val_na==0] %>%  
  .[ na.ns==0] %>% 
  .[ na.inc==0] %>%
  .[ dose.unq==0] %>%
  .[ all_dose_in_study==TRUE] %>%
  .[ dose_dup==0] %>% 
  .[ sample_size_all > 0]

setnames(dataContinuous, 
         c("dose_adjusted", "sample_size", "effect_val_adj", "effect_var_adj"), 
         c("doses", "ns", "means", "stdevs"))


In [17]:
head(dataContinuous)

study_id,chemical_id,dsstox_gsid,study_type,study_year,study_source,species,strain_group,admin_route,admin_method,...,effect_var_temp,id,id_count,na.ns,na.inc,dose.unq,dose_dup,all_dose_in_study,eff_val_na,sample_size_all
70,58739,34496,SUB,1997,opp_der,dog,beagle,Oral,Capsule,...,NA,70_176_122707_F,4,0,0,0,0,TRUE,0,4
70,58739,34496,SUB,1997,opp_der,dog,beagle,Oral,Capsule,...,NA,70_176_122712_M,4,0,0,0,0,TRUE,0,4
70,58739,34496,SUB,1997,opp_der,dog,beagle,Oral,Capsule,...,NA,70_176_122707_F,4,0,0,0,0,TRUE,0,4
70,58739,34496,SUB,1997,opp_der,dog,beagle,Oral,Capsule,...,NA,70_176_122712_M,4,0,0,0,0,TRUE,0,4
70,58739,34496,SUB,1997,opp_der,dog,beagle,Oral,Capsule,...,NA,70_176_122707_F,4,0,0,0,0,TRUE,0,4
70,58739,34496,SUB,1997,opp_der,dog,beagle,Oral,Capsule,...,NA,70_176_122712_M,4,0,0,0,0,TRUE,0,4


### Endpoints for Body Weight and Organ Weight 

In [20]:
dataContinuousBW <- dataContinuous[endpoint_type %in% c( "in life observation","organ weight") | endpoint_target=="body_weight"]

continuous_BW <- dataContinuousBW[ , list(id, doses, ns, means, stdevs) ] %>%
    aggregate( . ~ id, data = . , paste, collapse = ",")  %>% 
    data.table() %>%
    .[ , c("study_id", "endpoint_id", "tg_effect_id", "sex"):=tstrsplit(id, "_", fixed=TRUE)]

length(unique(continuous_BW$id))

## Write data to be sent to Notebook for BMDS processing 
write.csv(continuous_BW, "continuous_BW.csv")

[1] 5279

### Endpoints that are Not Body Weight and Organ Weight

In [21]:
dataContinuousNotBW <- dataContinuous[!(endpoint_type %in% c( "in life observation","organ weight") | endpoint_target=="body_weight")]

continuous_NotBW <- dataContinuousNotBW[ , list(id, doses, ns, means, stdevs) ] %>%
  aggregate( . ~ id, data = . , paste, collapse = ",")  %>% 
  data.table() %>%
  .[ , c("study_id", "endpoint_id", "tg_effect_id", "sex"):=tstrsplit(id, "_", fixed=TRUE)]

length(unique(continuous_NotBW$id))

## Write data to be sent to Notebook for BMDS processing
write.csv(continuous_NotBW,"continuous_NotBW.csv")

[1] 3989